In [16]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import transpile
from qiskit.circuit.library import TwoLocal
from qiskit_addon_cutting_custom.automated_cut_finding import *
from qiskit_addon_cutting_custom import *
# from circuit_knitting.cutting.automated_cut_finding import (
#     find_cuts,
#     OptimizationParameters,
#     DeviceConstraints,
# )
# from circuit_knitting.cutting import cut_wires, partition_problem

from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator, ErrorBudgetPartition

from hamlib.hamlib_snippets import *
from benchmark_utils import *
from gate_estimation.resource_analysis import *
from QASMBench.interface.qiskit import QASMBenchmark

workspace = Workspace (
   resource_id = "/subscriptions/e0d51919-2dda-4b7e-a8e7-77cc18580acb/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/QECBenchmarking",
   location = "uksouth"
)

estimator = MicrosoftEstimator(workspace)

In [24]:
c = build_Ising_hamiltonian(3, t=0.1, h=-1)

TypeError: ufunc 'ceil' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [20]:
c.draw(fold=-1)

┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               ┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───────────────┐                                                                                          

In [2]:
# # path to the root directory of QASMBench
# path = "QASMBench"
# # selected category for QASMBench
# category = "large" 
# # select only the circuits with the number of qubits in the list
# num_qubits_list = list(range(59, 64))
# # whether to remove the final measurement in the circuit
# remove_final_measurements = True
# # whether use qiskit.transpile() to transpile the circuits (note: must provide qiskit backend)
# do_transpile = False
# # arguments for qiskit.transpile(). backend should be provide at least
# transpile_args = {}
# bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, remove_final_measurements=remove_final_measurements, do_transpile=do_transpile, **transpile_args)
# circ = bm.get("qft_n63")

# cnot_number = bm.num_gates("qft_n63", instruction = 'cx')
# num_qubits_bm = bm.num_qubits("qft_n63")
# print(num_qubits_bm, cnot_number)

In [3]:
# two = TwoLocal(3, 'ry', 'cx', 'linear', reps=2, insert_barriers=False).decompose(reps=1)
# print(two)  # decompose the layers into standard gates

In [4]:
# of_data = read_openfermion_hdf5('hamlib/condensedmatter/tfim/tfim.hdf5', 'graph-1D-grid-nonpbc-qubitnodes_Lx-10_h-0')
# hamiltonian = qubitop_to_pauliop(of_data)
# circ_ham = build_hamiltonian(hamiltonian, t=1, num_timesteps=5)
# target_basis = ['rx', 'ry', 'rz', 'h', 'cx']
# circuit = transpile(circ_ham,
#                     basis_gates=target_basis, 
#                     optimization_level=1) 

In [2]:
circuit = build_Ising_hamiltonian(num_spins=25, h=-1, Dimension=(5,5))

number of time steps:  10


In [3]:
# Specify settings for the cut-finding optimizer
optimization_settings = OptimizationParameters(seed=111)

# Specify the size of the QPUs available
device_constraints = DeviceConstraints(qubits_per_subcircuit=circuit.num_qubits//2)

cut_circuit, metadata = find_cuts(circuit, optimization_settings, device_constraints)
print(
    f'Found solution using {len(metadata["cuts"])} cuts ({len(metadata["gate_cuts"])} gate {len(metadata["wire_cuts"])} wire cuts) with a sampling '
    f'overhead of {metadata["sampling_overhead"]}.'
)
# for cut in metadata["cuts"]:
#     print(f"{cut[0]} at circuit instruction index {cut[1]}")
# cut_circuit.draw("mpl", scale=0.8, fold=-1)

Found solution using 2900 cuts (2900 gate 0 wire cuts) with a sampling overhead of 75707401.0.


In [4]:
qc_w_ancilla = cut_wires(cut_circuit)
partitioned_problem = partition_problem(circuit=qc_w_ancilla)
subcircuit= partitioned_problem.subcircuits
subcircuits = [cuts_filter(s) for s in subcircuit.values()]
# subcircuits[0].draw("mpl", style="iqp", scale=0.5)

In [5]:
print(len(subcircuits))

3


In [8]:
# subcircuit[2].draw("mpl", style="iqp")

In [9]:
# qubits = []
# overhead = []
# for entanglement in range(3,60):
#     qubits.append(entanglement)
#     ent_circ = TwoLocal(entanglement, 'ry', 'cx', 'linear', reps=2, insert_barriers=False).decompose(reps=1)
#     # Specify settings for the cut-finding optimizer
#     optimization_settings = OptimizationParameters(seed=111)

#     # Specify the size of the QPUs available
#     device_constraints = DeviceConstraints(qubits_per_subcircuit=ent_circ.num_qubits//2)

#     cut_circuit, metadata = find_cuts(ent_circ, optimization_settings, device_constraints)
#     overhead.append(metadata["sampling_overhead"])
# plt.scatter(qubits, np.log(overhead))
# plt.show() 


In [6]:
params = estimator.make_params(num_items=1)
params.error_budget = 0.01
params.constraints.max_t_factories = 1
job = estimator.submit(circuit, input_params=params)
r = job.get_results()
Q = r['physicalCounts']['physicalQubits']
T = r['physicalCounts']['runtime']*1e-3
print('\n', 'total physical qubits required: ', Q, '\n', 'total runtime', T)
# total physical qubits required:  8276 
# total runtime 3498.0

.....
 total physical qubits required:  56148 
 total runtime 8309600.0


In [ ]:
# Q_approx = PhysicalParameters.make_beverland_et_al(circuit, error_budget=[0.01/3]*3).Q
# q = []
# sub_error_budget = [(0.01/20*e, 0.01-0.01/20*e) for e in range(1,20)]
# for (e1, e2) in sub_error_budget:
#     Q1 = PhysicalParameters.make_beverland_et_al(subcircuit[0], error_budget = [e1/3]*3).Q
#     Q2 = PhysicalParameters.make_beverland_et_al(subcircuit[1], error_budget = [e2/3]*3).Q 
#     q.append(max(Q1, Q2))

# plt.scatter([e1 for (e1,e2) in sub_error_budget], q, label='subcircuits')
# plt.plot([0, 0.01], [Q_approx, Q_approx], 'r--', label='original circuit')
# plt.legend()
# plt.xlabel('Error budget for the first subcircuit')
# plt.ylabel('Number of Physical qubits')
# plt.show()


In [ ]:
# Q_approx = PhysicalParameters.make_beverland_et_al(circuit, error_budget=[0.01/3]*3).Q
# q = []
# sub_error1 = []
# # sol = {}
# sol_q = Q_approx
# sol_e = None
# partition = error_budget_partition(0.01, len(subcircuits)*3)
# sub_error_budget = partition + [l[::-1] for l in partition]
# for e in sub_error_budget:
#     Q1 = PhysicalParameters.make_beverland_et_al(subcircuit[0], error_budget = [i/5000 for i in e[:3]]).Q
#     Q2 = PhysicalParameters.make_beverland_et_al(subcircuit[1], error_budget = [i/5000 for i in e[3:]]).Q 
#     m = max(Q1, Q2)
#     q.append(m)
#     sub_error1.append(sum(e[:3])/5000)
#     if sol_q > m:
#         sol_q = m
#         sol_e = e
        
# print(sol_q, sol_e)
# plt.scatter(sub_error1, q, label='subcircuits')
# plt.plot([0, 0.01], [Q_approx, Q_approx], 'r--', label='original circuit')
# plt.legend()
# # plt.ylim([min(q)-1000, Q_approx+1000])
# plt.xlabel('Error budget for the first subcircuit')
# plt.ylabel('Number of Physical qubits')
# plt.show()


In [7]:
quantum_runtime = []
physical_qubits = []
for idx, s in enumerate(subcircuits):
    params = estimator.make_params(num_items=1)
    params.items[0].constraints.max_t_factories = 1

    job = estimator.submit(s, input_params=params)
    r = job.get_results()
    quantum_runtime.append(r['physicalCounts']['runtime'])
    physical_qubits.append(r['physicalCounts']['physicalQubits'])
    print('\n', 'subcircuit-', idx+1, ': physical qubits--', r['physicalCounts']['physicalQubits'], 'runtime (in microsec)--', r['physicalCounts']['runtime']*1e-3)
print('quantum runtime total: ', sum(quantum_runtime)*1e-9, 'number of physical qubits: ', max(physical_qubits))

....
 subcircuit- 1 : physical qubits-- 11708 runtime (in microsec)-- 43758.0
....
 subcircuit- 2 : physical qubits-- 43270 runtime (in microsec)-- 3994278.8000000003
....
 subcircuit- 3 : physical qubits-- 43270 runtime (in microsec)-- 5650273.2
quantum runtime total:  9.688310000000001 number of physical qubits:  43270


In [11]:
r['logicalQubit']['physicalQubits']

722

In [15]:
r['physicalCounts']['breakdown']['logicalDepth']
r['physicalCounts']['breakdown']['algorithmicLogicalDepth']

23230

In [ ]:
# classical_runtime = metadata["sampling_overhead"]/1e14
# quantum_runtime = r1['physicalCounts']['runtime']*1e-9 + r2['physicalCounts']['runtime']*1e-9
# print(classical_runtime/86400, quantum_runtime)